In [175]:
import zipfile
import os
import pandas as pd
import numpy as np

DataFrame 불러오기

Data : ace 위성의 데이터, 국내 전파 센터 3곳(이전, 제주, 강릉)의 데이터

In [497]:
df = pd.read_csv("/home/competition/result/data/whole_df.csv")


df = df.drop("datetime",axis =1)
df = df.astype(float)


df.columns = ["datetime", "proton_density", "proton_temperature", "proton_speed","bx_gsm","by_gsm","bz_gsm","bt"]
df.drop(["datetime", "proton_density", "proton_temperature", "proton_speed"],axis =1,inplace=True)



In [ ]:
# 결측값 및 첨도, 왜도, describe를 통한 기본적 데이터 탐색
df.isnull().sum(),df.kurt(),df.skew(),df.describe()

In [ ]:

#이천, 제주, 강릉 데이터 불러오기
#Datatime 과 DOY 는 시간 데이터이므로 drop

df_K_ich = pd.read_csv("/home/competition/result/data/whole_df_K_ich.csv",index_col=False)
df_K_ich.drop(["Unnamed: 0","datetime", "DOY"],axis=1,inplace = True)
df_K_ich["sq"] = np.sqrt(df_K_ich["X"]**2 + df_K_ich["Y"]**2 + df_K_ich["Z"]**2)

df_K_jj = pd.read_csv("/home/competition/result/data/whole_df_K_jj.csv",index_col=False)
df_K_jj.drop(["Unnamed: 0","datetime", "DOY"],axis=1,inplace = True)
df_K_jj["sq"] = np.sqrt(df_K_jj["X"]**2 + df_K_jj["Y"]**2 + df_K_jj["Z"]**2)


df_K_gang = pd.read_csv("/home/competition/result/data/whole_df_K_gang.csv",index_col=False)
df_K_gang.drop(["Unnamed: 0","datetime", "DOY"],axis=1,inplace = True)
df_K_gang["sq"] = np.sqrt(df_K_gang["X"]**2 + df_K_gang["Y"]**2 + df_K_gang["Z"]**2)



df_whole = pd.concat([df,df_K_jj,df_K_ich,df_K_gang],axis =1)
pd.options.display.float_format = '{:.5f}'.format
df_whole.describe()

In [ ]:
# clip을 통한 상한선, 하한선 밖의 데이터 cliping
# 각 상한선 및 하한선은 IQR의 2.5배로 설정
Q3 = df_whole.quantile(0.75) 
Q1 = df_whole.quantile(0.25)
IQR = Q3 - Q1

lower_line = Q1-IQR*2.5
upper_line = Q3+IQR*2.5


df_whole.clip(lower = lower_line, upper = upper_line, axis=1, inplace = True)
df_whole


In [ ]:
# 위에서 데이터 탐색을 한 결과, 이상치가 많이 존재했으므로 이상치에 둔감한 QuantileTransformer 적용.
# 1000분위수로 적용함.
from sklearn.preprocessing import  QuantileTransformer

scaler_ = QuantileTransformer(output_distribution="normal", n_quantiles=1000)
scaler_.fit(df_whole)
X=scaler_.fit_transform(df_whole)
X = pd.DataFrame(X)

X, X.isnull().sum()

In [ ]:
# 시계열 데이터를 만들기 위해, 180개의 분 데이터(3시간)을 하나의 array로 설정
# 만약 데이터 중간에 null 값이 있다면 데이터 삭제
def make_dataset(data,window_size=180):
    feature_list = []
    null_list = []
    for i in range(0,len(data),180):
        do = True
        for a in range(i,i+180):
            if data.loc[a].isnull().any() == True:
                do = False
                break
        if do:
            feature_list.append(np.array(data.iloc[i:i+window_size]))
        else:
            null_list.append(i/180)
        
    return np.array(feature_list),null_list
X,null_list = make_dataset(X,180)
X,len(X)

In [ ]:
# target 값 불러오기
y = pd.read_csv("/home/competition/result/data/whole_y.csv",index_col = 0)
y = y.reset_index(drop = True)
y,len(y)

In [ ]:
# float 형태로 불러와 max, min 값 확인
y =y.drop(null_list)
y["0"] = y["0"].astype(float)
soft_max = y.max()
soft_min = y.min()
print(y.max(),y.min())
"""
y_scaler = MinMaxScaler()
y_scaler.fit(y)
y=y_scaler.fit_transform(y)
"""
y,len(y),y.max(),y.min()

In [ ]:
# train 과 valid 을 0.2 비율로 나누기
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,random_state =5)
x_train.shape, x_valid.shape,y_train

In [784]:
# 모델링
# 첫 layer는 LSTM 을 적용하고 나머지는 Dense Layer 적용
# LSTM 은 tanh, Dense 는 relu 적용
# 각 Dense layer는 he_normal 로 initalization
# regularizer 는 l2로 0.01 적용
# 이후 Dropout 0.2 적용
from keras.models import Sequential 
from keras.layers import Dense, Dropout 
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from keras.layers import LSTM, BatchNormalization 
import tensorflow as tf 
import keras
leaky_relu = tf.nn.leaky_relu 
model = Sequential() 
model.add(LSTM(256, input_shape=(x_train.shape[1], x_train.shape[2]),
               kernel_initializer='he_normal',activation = "tanh"
               ))

model.add(BatchNormalization()) 
model.add(Dense(512,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(Dropout(0.2))

model.add(BatchNormalization()) 
model.add(Dense(1024,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(Dropout(0.2))

model.add(BatchNormalization()) 
model.add(Dense(2048,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(Dropout(0.2))


model.add(BatchNormalization()) 
model.add(Dense(1024,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(Dropout(0.2))

model.add(BatchNormalization()) 
model.add(Dense(512,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(Dropout(0.2))

model.add(BatchNormalization()) 
model.add(Dense(256,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(Dropout(0.2))

model.add(BatchNormalization()) 
model.add(Dense(128,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(Dropout(0.2))

model.add(BatchNormalization()) 
model.add(Dense(64,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(Dropout(0.2))

model.add(BatchNormalization()) 
model.add(Dense(32,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(Dropout(0.2))

model.add(BatchNormalization()) 
model.add(Dense(16,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(Dropout(0.2))

model.add(BatchNormalization()) 
model.add(Dense(8,activation = leaky_relu,kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(Dropout(0.2)) 

model.add(Dense(1)) 
#model.add(Dense(soft_max - soft_min+1, activation = "softmax"))

In [ ]:
# optimizer 는 Adam, loss 는 mse 적용함
# batch size 는 128, epoch은 500 적용(early stop을 이용하여 학습 도중 마무리되게 설정)

import tensorflow.keras.backend as K
import os 
import tensorflow as tf 
import tensorflow as tf 
from tensorflow.keras.optimizers import RMSprop
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
adam = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) 
loss = "mse"
opt = adam#RMSprop(learning_rate=0.001, rho=0.9) 
model.compile(optimizer=opt, loss=loss, metrics=['mae']) 
early_stop = EarlyStopping(monitor='val_loss', patience=50)

model_path = 'model' 
filename = os.path.join(model_path, 'tmp_checkpoint.h5') 
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto') 
batch_size = 128
history = model.fit(x_train, y_train, epochs=500, 
                    batch_size=batch_size, 
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])

In [ ]:
# 예측 데이터 불러오기 및 전처리
pred_data = pd.read_csv("/home/competition/result/data/ACE_DSCOVR_Satellite/solar_wind_data_verif_2.csv")

pred_data = pred_data.drop(["time"], axis =1)
pred_data.columns = ["datetime","proton_density","proton_temperature","proton_speed","bx_gsm","by_gsm","bz_gsm","bt"]
pred_data.drop(["datetime", "proton_density", "proton_temperature", "proton_speed"],axis =1,inplace = True)


pred_data = pred_data.astype(float)


pred_jj = pd.read_csv("/home/competition/result/data/Ground_Geomagnetic_Data_Korea/Jeju/geomagnetic_field_data_jj_verif_2.csv")
pred_jj.drop(["day","time"],axis =1 , inplace = True)
pred_jj["sq"] = np.sqrt(pred_jj["X"]**2 + pred_jj["Y"]**2 + pred_jj["Z"]**2)

pred_ich = pd.read_csv("/home/competition/result/data/Ground_Geomagnetic_Data_Korea/Icheon/geomagnetic_field_data_ich_verif_2.csv")
pred_ich.drop(["day","time"],axis =1 , inplace = True)
pred_ich["sq"] = np.sqrt(pred_ich["X"]**2 + pred_ich["Y"]**2 + pred_ich["Z"]**2)


pred_gang = pd.read_csv("/home/competition/result/data/Ground_Geomagnetic_Data_Korea/Gangneung/geomagnetic_field_data_gn_verif_2.csv")
pred_gang.drop(["day","time"],axis =1 , inplace = True)
pred_gang["sq"] = np.sqrt(pred_gang["X"]**2 + pred_gang["Y"]**2 + pred_gang["Z"]**2)



pred_data_whole = pd.concat([pred_data,pred_jj,pred_ich, pred_gang], axis=1)

for a in pred_data_whole.columns:
    print(a,pred_data_whole[a].kurt(),pred_data_whole[a].skew())


#pred_data_whole.clip(lower = lower_line, upper = upper_line, axis=1,inplace = True)
print(pred_data_whole.isnull().sum())
pred_data_whole = scaler_.fit_transform(pred_data_whole)
pred_data_whole = pd.DataFrame(pred_data_whole)
pred_data_whole = pred_data_whole.interpolate(method = "pad",limit =5000)
for a in pred_data_whole.columns:
    print(a,pred_data_whole[a].kurt(),pred_data_whole[a].skew())

pred_data_whole.describe()

In [794]:
def pred_make_dataset(data,window_size=180):
    feature_list = []
    
    for i in range(0,len(data),180):
        
            feature_list.append(np.array(data.iloc[i:i+window_size]))
        
    return np.array(feature_list)

In [797]:
# 예측
pred = pred_make_dataset(pred_data_whole,180)
from keras.models import load_model
model = load_model('model/tmp_checkpoint_0-Copy1.61_layer256~2048_mse_128_last.h5', custom_objects={'root_mean_squared_error': root_mean_squared_error})
prediction = model.predict(pred)
#result = y_scaler.inverse_transform(pd.DataFrame(prediction))

result = pd.DataFrame(prediction).interpolate(method = "pad",limit =9)
res = result.iloc[240:].round(0)
res.to_csv("result_lastest.csv",index=False)

In [679]:
res_list = []
for a in range(len(res)):
    now=res.iloc[a,:].to_list()
    res_list.append((now.index(max(now))))
res_list = pd.DataFrame(res_list)
res_list.to_csv(f"result_soft_max_{loss}_{opt}_{batch_size}_lasttest.csv",index=False)